# Trackster-level graph net with PU

Question: does including the neighborhood provide a benefit over the pairwise classification?

In [1]:
import numpy as np
import torch
import sys

import torch.nn as nn
from torch.optim import SGD
from torch_cluster import knn_graph
from torch.optim.lr_scheduler import CosineAnnealingLR

from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

from reco.model import DynamicEdgeConvBlock

from reco.training import precision_recall_curve, roc_auc
from reco.loss import GraphClassificationLoss
from reco.datasetPU import TracksterGraph


ds_name = "CloseByTwoPion"

data_root = "data"
raw_dir = f"/Users/ecuba/data/{ds_name}"

# data_root = "/mnt/ceph/users/ecuba/processed"
# raw_dir = f"/mnt/ceph/users/ecuba/{ds_name}"

In [2]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
def create_mask(data):
    # extract the focus feature
    data.mask = (1 - data.x[:,0]).type(torch.bool)
    return data

def knn_transform(data):
    # pos coordinates are on position 3:6
    data.edge_index = knn_graph(data.pos, k=4, loop=False)
    return data

def prepend_pos(data):
    # GravNet: add S to the front
    data.x = torch.hstack((data.pos, data.x))
    return data

transforms = T.Compose([prepend_pos])

In [19]:
ds = TracksterGraph(
    ds_name,
    data_root,
    raw_dir,
    N_FILES=20,
    radius=100,
    bigT_e_th=75,
    # transform=transforms
    # pileup=True,
)

# ds.processed_file_names
ds.data

Data(x=[101464, 39], y=[101464], pos=[101464, 3], e=[101464], shared_e=[101464], node_index=[101464])

In [5]:
len(ds.data.y)

101464

In [6]:
ds_size = len(ds)
test_set_size = ds_size // 10
train_set_size = ds_size - test_set_size
train_set, test_set = random_split(ds, [train_set_size, test_set_size])
print(f"Train graphs: {len(train_set)}, Test graphs: {len(test_set)}, total: {ds_size}")

train_dl = DataLoader(train_set, batch_size=16, shuffle=True)
test_dl = DataLoader(test_set, batch_size=16, shuffle=True)

Train graphs: 2864, Test graphs: 318, total: 3182


In [7]:
class TracksterGraphNet(nn.Module):
    def __init__(self, input_dim, output_dim=1, dropout=0.2, S=3):
        super(TracksterGraphNet, self).__init__()
        
        hdim1 = 64
        hdim2 = 64
        hdim3 = 64

        hdim_fc = 256

        self.edgeconv1 = DynamicEdgeConvBlock(input_dim, hdim1)
        self.edgeconv2 = DynamicEdgeConvBlock(hdim1, hdim2)
        self.edgeconv3 = DynamicEdgeConvBlock(hdim2, hdim3)
        
        # Edge features from node embeddings for classification        
        self.nodenetwork = nn.Sequential(
            nn.Linear(hdim3, hdim_fc),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hdim_fc, 3),
            nn.ReLU(),
            nn.Linear(3, output_dim),
            nn.Sigmoid()
        )

    def forward(self, X):
        H1 = self.edgeconv1(X)
        H2 = self.edgeconv2(H1)
        H3 = self.edgeconv3(H2)
        return self.nodenetwork(H3).squeeze(-1)

In [8]:
model = TracksterGraphNet(input_dim=ds.data.x.shape[1])
epochs = 10
model_path = f"models/TracksterGraphNet.DGCNN.ns.{epochs}e-{ds_name}.r{ds.RADIUS}.e{ds.bigT_e_th}.f{ds.N_FILES}.pt"

In [9]:
def train_graph_classification(model, device, optimizer, loss_func, train_dl):
    train_loss = []
    model.train()

    for data in train_dl:
        data = data.to(device)
        optimizer.zero_grad()

        seg_pred = model(data.x)
        loss = loss_func(data, seg_pred)
        loss.backward()

        optimizer.step()

        train_loss.append(loss.item())

    return np.mean(train_loss)

In [10]:
@torch.no_grad()
def test_graph_classification(model, device, loss_func, test_dl):
    test_loss = []
    model.eval()
    for data in test_dl:
        data = data.to(device)
        seg_pred = model(data.x)
        loss = loss_func(data, seg_pred)
        test_loss.append(loss.item())

    return np.mean(test_loss)

In [11]:
%%script echo skipping
# alpha - percentage of negative edges
loss_func = GraphClassificationLoss()

model = model.to(device)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=1e-5)

for epoch in range(epochs):

    train_loss = train_graph_classification(
        model,
        device,
        optimizer,
        loss_func,
        train_dl
    )
    
    scheduler.step()

    if epoch % 1 == 0:
        test_loss = test_graph_classification(model, device, loss_func, test_dl)
        print(
            f"Epoch {epoch}\t train loss: {train_loss:.3f}\t test loss: {test_loss:.3f}",
            file=sys.stderr
        )
        
torch.save(model.state_dict(), model_path)
print(model_path)



skipping


In [12]:
# %%script echo skipping
model.load_state_dict(torch.load(
    model_path,
    map_location=device
))
model = model.to(device)

In [13]:
%%script echo skipping
print(roc_auc(model, device, test_dl))
precision_recall_curve(model, device, test_dl, focus_metric="fbeta", beta=0.5, step=3, truth_threshold=0.5)

skipping


## Eval

In [14]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
from reco.data import get_event_data
from reco.evaluation import model_evaluation

# from reco.dummy import DummyPleaser

file_name = f"{raw_dir}/new_ntuples_15101852_0.root"
cluster_data, trackster_data, simtrackster_data, assoc_data = get_event_data(file_name)

In [15]:
result = model_evaluation(
    cluster_data,
    trackster_data,
    simtrackster_data,
    assoc_data,
    model,
    decision_th=0.5,
    radius=100,
    max_events=20,
    bigT_e_th=75,
    graph=True,
)

Event 0:
	clue3d_to_sim:	P: 0.988 R: 0.229 F: 0.594
	target_to_sim:	P: 0.981 R: 0.686 F: 0.903
	reco_to_sim:	P: 0.974 R: 0.678 F: 0.896
	|S| = 2 |T| = 2 |R| = 2
Event 1:
	No data
Event 2:
	clue3d_to_sim:	P: 1.000 R: 0.396 F: 0.766
	target_to_sim:	P: 0.999 R: 0.814 F: 0.956
	reco_to_sim:	P: 0.973 R: 0.795 F: 0.931
	|S| = 2 |T| = 2 |R| = 2
Event 3:
	clue3d_to_sim:	P: 0.995 R: 0.383 F: 0.754
	target_to_sim:	P: 0.991 R: 0.782 F: 0.941
	reco_to_sim:	P: 0.979 R: 0.767 F: 0.928
	|S| = 2 |T| = 2 |R| = 2
Event 4:
	clue3d_to_sim:	P: 0.998 R: 0.250 F: 0.625
	target_to_sim:	P: 0.996 R: 0.838 F: 0.960
	reco_to_sim:	P: 0.987 R: 0.828 F: 0.951
	|S| = 2 |T| = 2 |R| = 2
Event 5:
	clue3d_to_sim:	P: 0.996 R: 0.551 F: 0.858
	target_to_sim:	P: 0.995 R: 0.839 F: 0.959
	reco_to_sim:	P: 0.963 R: 0.804 F: 0.926
	|S| = 2 |T| = 2 |R| = 2
Event 6:
	clue3d_to_sim:	P: 0.992 R: 0.219 F: 0.582
	target_to_sim:	P: 0.992 R: 0.727 F: 0.925
	reco_to_sim:	P: 0.941 R: 0.668 F: 0.870
	|S| = 2 |T| = 2 |R| = 2
Event 7:
	No dat

## Results

Using alpha=balance seems to converge faster (100 epochs rather than 200)
- maybe just sticking to 0.25? (works just fine)

Hyperparams
- (64, 128, noskip), alpha=balance, roc_auc = 0.9733
- (64, 128, 256fc, noskip, 200e), alpha=1-balance, roc_auc = 0.9771
- (64, 128, 256fc, noskip, 200e), alpha=balance, roc_auc = 0.981
- (64, 128, 256fc, noskip, 100e), alpha=0.25, roc_auc = 0.9796

In [16]:
import torch.onnx

onnx_filepath = f"onnx/TracksterGraphNet.mask.64.128.256.ns.{epochs}e-{ds_name}.{ds.RADIUS}.{ds.SCORE_THRESHOLD}.{ds.N_FILES}f.onnx"

torch.onnx.export(
    model,                          # model to be exported
    (ds[0].x),    # example input (add batch dimension)
    onnx_filepath,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['features'],      # the model's input names
    output_names=['output'],    # the model's output names
    dynamic_axes={              # variable length axes
        'features' : {0 : 'batch_size'},    
        'output' : {0 : 'batch_size'}
    }
)

UnsupportedOperatorError: ONNX export failed on an operator with unrecognized namespace torch_cluster::knn. If you are trying to export a custom operator, make sure you registered it with the right domain and version.